In [1]:
!pip install pytorch_lightning

     |████████████████████████████████| 256kB 2.9MB/s 
     |████████████████████████████████| 829kB 8.9MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=b88adc7e011909fab4011452f58f717c54aaab472dcc49d20f7e1c3a8caf520c
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [2]:
# Mount your personal google drive
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
primary_directory = '/content/drive/My Drive/KBP_Challenge/'
%cd /content/drive/My\ Drive/KBP_Challenge
%ls

/content/drive/My Drive/KBP_Challenge
checkpoints/  data/  lightning_logs/  options/  provided_code/  results/  src/


In [0]:
import torch
import pytorch_lightning as pl
from src.pix2pix_model import Pix2PixModel
from options.train_options import TrainOptions

In [7]:
args = ['--batchSize', '1',
        '--which_model_netG', 'unet_128_3d',
        '--which_direction', 'AtoB',
        '--input_nc', '1',
        '--lambda_A', '100']

opt = TrainOptions().parse(args)
checkpoint_number = 14
checkpoint_file = '{}/checkpoints/epoch={}.ckpt'.format(primary_directory, checkpoint_number)
dataset_dir = '{}/data'.format(primary_directory)
model = Pix2PixModel.load_from_checkpoint(checkpoint_file, opt, dataset_dir, stage='hold-out-tests')

trainer = pl.Trainer(max_epochs=1)
trainer.test(model)

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
dataroot: None
dataset_mode: unaligned
display_freq: 200
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 128
epoch_count: 1
fineSize: 128
gpu_ids: []
init_type: normal
input_nc: 1
isTrain: True
lambda_A: 100.0
lambda_B: 10.0
lambda_identity: 0.5
loadSize: 128
lr: 0.0002
lr_decay_iters: 50
lr_policy: lambda
max_dataset_size: inf
model: pix2pix
nThreads: 2
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_dropout: False
no_flip: False
no_html: False
no_img: False
no_lsgan: False
norm: batch_3d
nwf: 64
output_nc: 1
phase: train
pool_size: 50
print_freq: 200
recon_loss: l1
resize_or_crop: resize_and_crop
save_epoch_freq: 5
save_latest_freq: 5000
serial_batches: False
update_html_freq: 1000
wasserstein: False
which_direction: AtoB
which_epoch: latest
which_model_netD: n_layers_3d
which_model_netG: unet_128_3d
which_mo

NameError: ignored